In [228]:
%matplotlib widget

import requests
import json
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as st
from scipy.stats import linregress
from collections import Counter

### GET DATA FROM pulldata FILE OUTPUT

In [220]:
#get csv files
#study data files
multi_city = "multi_restaurant_data.csv"

#set data to data frame
multi_df = pd.read_csv(multi_city)

In [213]:
#Run this code to set dataframe by defined city otherwise skip this cell
multi_df = multi_df.loc[multi_df["City"]=="Perth"]
multi_df

,Restaurant ID,Name,Cuisine,Locality,City,Rating,Total Number Ratings,Delivery,Latitude,Longitude
40,16598837,Run Amuk,Fast Food,"Orient Street, South Fremantle",Perth,4.9,341,No,-32.072277,115.753065
10,16597513,Pacific Rim Mix Plate,"Hawaiian, Japanese",Applecross,Perth,4.9,213,No,-32.021900,115.832287
70,16598976,Marumo,"Japanese, Seafood, Modern Australian",Nedlands,Perth,4.9,141,No,-31.980216,115.796659
3,16598431,Tuck Shop Cafe,"Modern Australian, Cafe Food",Northbridge,Perth,4.8,799,No,-31.946736,115.862108
23,16598168,Nobu Perth,"Japanese, Sushi",Burswood,Perth,4.8,663,No,-31.960526,115.894025
...,...,...,...,...,...,...,...,...,...,...
41,16600326,Mary Street Bakery,Bakery,Highgate,Perth,4.0,518,No,-31.938302,115.869470
77,16601688,Tropico,"Tapas, Modern Australian, Pizza",North Beach,Perth,4.0,302,No,-31.856008,115.754412
58,16599918,Jamie's Italian,Italian,Perth CBD,Perth,3.8,1473,No,-31.951720,115.858058
86,16598361,The Wild Fig Cafe,"Modern Australian, Cafe Food",Scarborough,Perth,3.3,709,No,-31.897999,115.757109


In [222]:
#This Section shows the top 10 Highest Rated Restaurant in the Data Frame
#Sorted by Rating then Total Number of Ratings
#multi_df.head()
multi_df.sort_values(by=["Rating","Total Number Ratings"] ,inplace=True, ascending = False)
multi_df.dropna(inplace=True)
multi_df.head(10)

,Restaurant ID,Name,Cuisine,Locality,City,Rating,Total Number Ratings,Delivery,Latitude,Longitude
129,16572612,Vue de monde,"Australian, Contemporary",CBD,Melbourne,4.9,986,No,-37.818954,144.957934
163,16582263,Gelato Messina,"Ice Cream, Desserts",Fitzroy,Melbourne,4.9,835,No,-37.801891,144.983674
103,16583098,Lune Croissanterie,"Bakery, French",Fitzroy,Melbourne,4.9,753,No,-37.796026,144.979924
101,16585905,Tipo 00,Italian,CBD,Melbourne,4.9,721,No,-37.813528,144.961973
137,17881527,Dexter,"American, BBQ",Preston,Melbourne,4.9,683,No,-37.736196,145.004456
191,16583248,Pidapipó,"Ice Cream, Desserts",Carlton,Melbourne,4.9,525,No,-37.798331,144.967186
215,16558798,Quay,Modern Australian,Circular Quay,Sydney,4.9,452,No,-33.858029,151.209970
102,16577212,Beatrix,"Coffee and Tea, Bakery",North Melbourne,Melbourne,4.9,384,No,-37.802367,144.944475
136,16578793,Patricia Coffee Brewers,Coffee and Tea,CBD,Melbourne,4.9,364,No,-37.814714,144.958260
233,16562433,Cow & The Moon,"Ice Cream, Desserts, Coffee and Tea",Enmore,Sydney,4.9,356,No,-33.899000,151.172029


In [224]:
#get all possible cuisine in dataframe and set them in a list
cuisine_list = []
#loop through rows
#get all cuisine types store them in temp list
for index, row in multi_df.iterrows():
    cuiser = row["Cuisine"]
    cuiser = cuiser.split(", ")
    for cuisine in cuiser:
        cuisine_list.append(cuisine)

In [245]:
#count the duplicated cuisine in the list
cuisine_count = dict(Counter(cuisine_list))
cuis_df = pd.DataFrame({"Cuisine Count":cuisine_count})
cuis_df.sort_values(by=['Cuisine Count'], inplace=True, ascending = False)
cuis_df = cuis_df.head(6)
print(cuis_df)
#drop the coffee and tea cause it's already included in cafe food
cuis_df = cuis_df.drop(cuis_df.index[1])

#get the Top 1 cuisine
top_cuis = cuis_df.head(1)
for topcuisine in top_cuis.index:
    top_cuis = topcuisine
top_cuis

                   Cuisine Count
Cafe Food                    236
Coffee and Tea               216
Modern Australian            184
Japanese                     118
Asian                         70
Italian                       70


'Cafe Food'

In [217]:
cuis_df.plot(kind = "bar",align="center",rot=360)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### CAFE FOOD or TOP RATED CUISINE
*This Section the DataFrame will show all the restaurant that serves Cafe Food and Drop all restaurant that doesn't.

In [247]:
index_to_drop = []
#loop through all rows of dataset
for index, row in multi_df.iterrows():
    #split the cuisine cell
    cuiser = row["Cuisine"]
    cuiser = cuiser.split(", ")
    cf = False
    #check and get restaurants that serves cafe food then drop all restaurant that doesn't
    for cuisine in cuiser:
        if cuisine == top_cuis:
            cf = True
    if cf == False:
        index_to_drop.append(index)
new_multi_df = multi_df.drop(index_to_drop)
new_multi_df

,Restaurant ID,Name,Cuisine,Locality,City,Rating,Total Number Ratings,Delivery,Latitude,Longitude
301,18441945,Jacaranda Coffee Lane,"Fusion, Coffee and Tea, Desserts, Cafe Food",West End,Brisbane,4.9,147,No,-27.479948,153.013047
346,16591327,Bunker Coffee,"Cafe Food, Coffee and Tea",Milton,Brisbane,4.9,108,No,-27.469477,153.006600
952,16605025,Fuel Coffee + Food,"Coffee and Tea, Cafe Food",Geelong CBD,Geelong,4.9,71,No,-38.145911,144.360734
106,18423923,Humble Rays,"Coffee and Tea, Cafe Food, Asian Fusion",Carlton,Melbourne,4.8,829,No,-37.804818,144.961554
3,16598431,Tuck Shop Cafe,"Modern Australian, Cafe Food",Northbridge,Perth,4.8,799,No,-31.946736,115.862108
...,...,...,...,...,...,...,...,...,...,...
796,17874757,Brennans,"Healthy Food, Cafe Food",Newcastle,Newcastle,0.0,0,No,-32.973716,151.644354
799,16611151,Conroy Bradley Pistol Club,"Coffee and Tea, Cafe Food",Newcastle,Newcastle,0.0,0,No,-32.926679,151.777411
897,18249070,All Good Things Organic,Cafe Food,Wollongong,Wollongong,0.0,0,No,-34.426059,150.892028
931,19206662,View Cafe,Cafe Food,Advancetown,Advancetown,0.0,0,No,-28.050435,153.279384
